# Глубокое обучение и обработка естественного языка

## Домашняя работа №3

1. Загрузить набор данных Spam Or Not Spam (или любой другой, какой вам нравится)
2. Обучить модели и сравнить различные способы векторизации с помощью внутренней оценки (intrinsic):

  *   Word2Vec SkipGram / CBOW (параметр sg в gensim.models.word2vec.Word2Vec) - 3 балла
  *   fastText (можно взять в gensim, или в fasttext как на семинаре) - 2 балла

3. Обучить на полученных векторах модели LogisticRegression и сравнить качество на отложенной выборке - 2 балла
4. Обеспечена воспроизводимость решения: зафиксированы random_state, ноутбук воспроизводится от начала до конца без ошибок - 2 балла
5. Соблюден code style на уровне pep8 и On writing clean Jupyter notebooks - 1 балл

**Примечания**: \


*   Для получения более качественных эмбеддингов стоит предварительно сделать предобработку корпуса - отсеять стоп-слова, провести нормализацию и тп. Предобработка рассматривалась в первой лекции/семинаре
*   В данном случае под intrinsic оценкой подразумевается просто использование методов most_similar, doesnt_match. Однако, если есть желание, можно измерить косинусное расстояние между отдельными парами слов и проверить, есть ли корреляция с корпусами для intrinsic-оценки, которые обсуждались на семинаре



In [ ]:
# установка spaCy
!pip install -U spacy

# English pipeline в spaCy
!python3 -m spacy download en_core_web_sm

In [7]:
# подключение библиотек
import numpy as np
import pandas as pd
import spacy
import gensim.models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [8]:
# загрузка
from google.colab import files

uploaded_spam = files.upload()
uploaded_simlex = files.upload()

Saving spam_or_not_spam.csv to spam_or_not_spam.csv


Saving SimLex-999.txt to SimLex-999.txt


In [24]:
# функция, возвращает векторизированное предложение
def get_vector(model, sentence):
  sentence_vector = []

  if len(sentence) == 0:
    # Пустые предложения заполним их одним <unk> словом
    token_vector = np.zeros(256)
    sentence_vector.append(token_vector)
  else:
    for token in sentence:
      try:
        token_vector = model.wv[token]
      except KeyError as e:
        # Случай <unk> слова
        token_vector = np.zeros(256)
      finally:
        sentence_vector.append(token_vector)

  return np.mean(sentence_vector, axis=0)


# функция, возвращает векторизированные выборки
def vectorize_train_test(model, X_train, X_test, vector_size=256):
  X_train_vectorized = np.zeros((X_train.shape[0], vector_size))
  for index, sentence in enumerate(X_train):
    X_train_vectorized[index] = get_vector(model, sentence)

  X_test_vectorized = np.zeros((X_test.shape[0], vector_size))
  for index, sentence in enumerate(X_test):
    X_test_vectorized[index] = get_vector(model, sentence)

  return X_train_vectorized, X_test_vectorized

# функция, классификация текстов LogisticRegression
def texts_classify(X_train, y_train, X_test, y_test):
  logereg = LogisticRegression()
  logereg.fit(X_train, y_train)
  y_pred = logereg.predict(X_test)

  report = classification_report(y_test, y_pred, output_dict=True)
  return y_pred, report


"""
Функция, считает:
 - среднюю вероятность схожести с соседом
 - среднее косинусное расстояние между схожими словами
 - среднее значение similarity rating по SimLex-999 для примеров, найденных в датасете
"""
def calculate_metrics(model, simlex_data):
  words = model.wv.key_to_index.keys()
  mean_proba = 0
  mean_distance = 0
  mean_sim_rank = 0
  count = 0

  for word in words:
    candidate = model.wv.most_similar(positive=[word], topn=1)[0]
    mean_distance += model.wv.distance(candidate[0], word)
    mean_proba += candidate[1]
    simlex_pair = simlex_data[(simlex_data['word1'] == word) & (simlex_data['word2'] == candidate[0])]

    if simlex_pair.shape[0] != 0:
      mean_sim_rank += float(simlex_pair['SimLex999'])
      count += 1

  return mean_proba / len(words), mean_distance / len(words), (mean_sim_rank + 1) / (count + 1)

### 1. Разведочный анализ

In [10]:
df = pd.read_csv('spam_or_not_spam.csv')
df

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0
...,...,...
2995,abc s good morning america ranks it the NUMBE...,1
2996,hyperlink hyperlink hyperlink let mortgage le...,1
2997,thank you for shopping with us gifts for all ...,1
2998,the famous ebay marketing e course learn to s...,1


In [11]:
# типы данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   2999 non-null   object
 1   label   3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [12]:
# пропуски в данных
df.isna().sum()

email    1
label    0
dtype: int64

In [13]:
df = df.dropna()

In [14]:
# соотношение классов
df['label'].value_counts()

0    2500
1     499
Name: label, dtype: int64

### 2. Сравнение Word2Vec SkipGram/CBOW и FastText


In [15]:
# сводная таблица
index = ['SkipGram', 'CBOW', 'FastText']
columns = ['Logreg_acc']

res_df = pd.DataFrame([[0]] * 3, index=index, columns=columns)
res_df

,Logreg_acc
SkipGram,0
CBOW,0
FastText,0


#### Word2Vec SkipGram / CBOW
Для Word2Vec необходима предварительная обработка текста

##### Нормализация, токенизация и лемматизация

In [16]:
nlp = spacy.load("en_core_web_sm")

df['email_cleaned'] = df['email'].apply(
    lambda x: [token.lemma_.lower() for token in nlp(x) if
      not token.is_stop
      and not token.is_punct
      and not token.is_digit
      and not token.like_email
      and not token.like_num
      and not token.is_space
    ]
  )

<ipython-input-16-1edc3b734757>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['email_cleaned'] = df['email'].apply(


После нормализации в поле 'email_cleaned' появились пустые предложения. При векторизации предложений они будут заполнены одним \<unk> словом.

In [17]:
# тренеровочная и тестовая выборки
X_train, X_test, y_train, y_test = train_test_split(df['email_cleaned'], df['label'], random_state=2023)

##### Skip-gram

In [18]:
skip_gram = gensim.models.Word2Vec(
    sentences=X_train,
    vector_size=256,
    window=7,
    min_count=2,
    sg=1,
    hs=0,
    negative=5,
    epochs=25,
    seed=2023,
)

In [25]:
# векторизация предложений
X_train_vectorized, X_test_vectorized = vectorize_train_test(skip_gram, X_train, X_test)

# классификация текстов
y_pred, report = texts_classify(X_train_vectorized, y_train, X_test_vectorized, y_test)
res_df['Logreg_acc'][0] = report['accuracy']

##### CBOW

In [26]:
cbow = gensim.models.Word2Vec(
    sentences=df['email_cleaned'].to_numpy(),
    vector_size=256,
    window=7,
    min_count=10,
    sg=0,
    hs=0,
    negative=5,
    epochs=25,
    seed=2023,
)

In [27]:
# векторизация предложений
X_train_vectorized, X_test_vectorized = vectorize_train_test(cbow, X_train, X_test)

# классификация текстов
y_pred, report = texts_classify(X_train_vectorized, y_train, X_test_vectorized, y_test)
res_df['Logreg_acc'][1] = report['accuracy']

#### FastText
Для FastText необходима предварительная обработка текста. При этом, чтобы воспользоваться преимуществом FastText (формирование n-грамм), откажемся от проведения лемматизации поля 'email'

##### Нормализация, токенизация

In [28]:
nlp = spacy.load("en_core_web_sm")

df['email_tokenized'] = df['email'].apply(
    lambda x: [str(token).lower() for token in nlp(x) if
      not token.is_stop
      and not token.is_punct
      and not token.is_digit
      and not token.like_email
      and not token.like_num
      and not token.is_space
    ]
  )

<ipython-input-28-10d62511b181>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['email_tokenized'] = df['email'].apply(


##### Обучение модели

In [29]:
fasttext = gensim.models.FastText(
    vector_size=256,
    window=7,
    min_count=1
    )
fasttext.build_vocab(corpus_iterable=df['email_tokenized'])
fasttext.train(
    corpus_iterable=df['email_tokenized'],
    total_examples=len(df['email_tokenized']),
    epochs=25
    )

(8854127, 9925250)

In [30]:
# векторизация предложений
X_train_vectorized, X_test_vectorized = vectorize_train_test(fasttext, X_train, X_test)

# классификация текстов
y_pred, report = texts_classify(X_train_vectorized, y_train, X_test_vectorized, y_test)
res_df['Logreg_acc'][2] = report['accuracy']

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### 3. Оценка качества



##### Accuracy моделей
Оценка качества построенных embeddings по результату решаемой задачи


In [31]:
res_df

,Logreg_acc
SkipGram,0.984000
CBOW,0.976000
FastText,0.973333


##### Intristic-оценка
Оценивается качество построенных embeddings по распределению векторов в пространстве. Здесь считается средняя вероятность похожего слова для каждого слова корпуса и среднее косинусное расстояние для таких пар слов.

In [32]:
simlex = pd.read_csv('SimLex-999.txt', sep='\t')

In [33]:
print("Skip-gram:\n\t \
      mean_proba = {:.3f}\n\t \
      mean_cos_dist = {:.3f}\n\t \
      mean_sim_rank = {:.3f}".format(*calculate_metrics(skip_gram, simlex)))
print("CBOW:\n\t \
      mean_proba = {:.3f}\n\t \
      mean_cos_dist = {:.3f}\n\t \
      mean_sim_rank = {:.3f}".format(*calculate_metrics(cbow, simlex)))
print("FastText:\n\t \
      mean_proba = {:.3f}\n\t \
      mean_cos_dist = {:.3f}\n\t \
      mean_sim_rank = {:.3f}".format(*calculate_metrics(fasttext, simlex)))

Skip-gram:
	       mean_proba = 0.743
	       mean_cos_dist = 0.257
	       mean_sim_rank = 1.000
CBOW:
	       mean_proba = 0.652
	       mean_cos_dist = 0.348
	       mean_sim_rank = 3.578
FastText:
	       mean_proba = 0.911
	       mean_cos_dist = 0.089
	       mean_sim_rank = 0.815


In [34]:
!pip freeze > requirements.txt